In [33]:
import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.batchnorm1 = nn.BatchNorm2d(128)
        self.batchnorm2 = nn.BatchNorm2d(512)

        self.dropout = nn.Dropout2d(p=0.5)

        self.fc4 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.batchnorm1(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.batchnorm2(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        return x

In [20]:
data_directory = './data/Train-Test Set'

transform = transforms.Compose([
    transforms.Resize((640, 480)),
    transforms.ToTensor(),
])

custom_dataset = ImageFolder(root=data_directory, transform=transform)

In [21]:
torch.manual_seed(42)

In [22]:
batch_size = 32
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [114]:
model2 = Sequential()

model2.add(Conv2D(64, (2, 2), activation='relu', input_shape=(640, 480, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(.2))
model2.add(Conv2D(128, (2, 2), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(.2))
model2.add(Conv2D(256, (2, 2), activation='relu'))
# model2.add(Conv2D(256, (2, 2), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((3, 3)))
model2.add(Dropout(.2))
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [115]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [116]:
model2.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_72 (Conv2D)          (None, 639, 479, 64)      832       
                                                                 
 max_pooling2d_60 (MaxPooli  (None, 319, 239, 64)      0         
 ng2D)                                                           
                                                                 
 dropout_45 (Dropout)        (None, 319, 239, 64)      0         
                                                                 
 conv2d_73 (Conv2D)          (None, 318, 238, 128)     32896     
                                                                 
 batch_normalization_30 (Ba  (None, 318, 238, 128)     512       
 tchNormalization)                                               
                                                                 
 max_pooling2d_61 (MaxPooli  (None, 159, 119, 128)   

In [117]:
train_datagen = ImageDataGenerator(
    # rescale=1./255,
)
test_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    './data/Train-Test Set/Training',
    target_size=(640, 480),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    './data/Train-Test Set/Validation',
    target_size=(640, 480),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    './data/Train-Test Set/Testing',
    target_size=(640, 480),
    batch_size=32,
    class_mode='binary'
)

Found 115 images belonging to 2 classes.
Found 77 images belonging to 2 classes.
Found 48 images belonging to 2 classes.


In [118]:
model2.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

C:\Users\user\AppData\Local\Temp\ipykernel_22488\1707472198.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(


Epoch 1/10
4/4 [==============================] - 45s 13s/step - loss: 684.3063 - accuracy: 0.4261 - val_loss: 476.4276 - val_accuracy: 0.5844
Epoch 2/10
4/4 [==============================] - 37s 9s/step - loss: 359.8405 - accuracy: 0.5826 - val_loss: 736.7025 - val_accuracy: 0.5844
Epoch 3/10
4/4 [==============================] - 35s 9s/step - loss: 269.8095 - accuracy: 0.4000 - val_loss: 1885.3149 - val_accuracy: 0.5844
Epoch 4/10
4/4 [==============================] - 38s 11s/step - loss: 177.1110 - accuracy: 0.4783 - val_loss: 853.3491 - val_accuracy: 0.5844
Epoch 5/10
4/4 [==============================] - 38s 11s/step - loss: 119.0475 - accuracy: 0.5565 - val_loss: 1293.7122 - val_accuracy: 0.5844
Epoch 6/10


KeyboardInterrupt: 

In [ ]:
model2.save('cnn_only.h5')

In [100]:
from sklearn.metrics import confusion_matrix
import itertools

predictions = model2.predict_generator(test_generator)
y_pred = np.round(predictions).astype(int).flatten()  # Convert probabilities to class labels
print(y_true, y_pred)
y_true = test_generator.classes
cm = confusion_matrix(y_true, y_pred)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

plot_confusion_matrix(cm, classes=validation_generator.class_indices.keys())

plt.show()

C:\Users\user\AppData\Local\Temp\ipykernel_22488\1442634390.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model2.predict_generator(test_generator)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1] [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]


ValueError: Found input variables with inconsistent numbers of samples: [48, 96]